# Plot number of bees scenting away from queen (0,0)

Goal: Plot proportion of correctly scenting bees / total scenting bees.

In [2]:
# Imports
import json
import numpy as np
import plotly.plotly as py
import matplotlib as mpl
from matplotlib import cycler
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import os
import glob2
import re
import random
from sklearn.preprocessing import normalize
import math


Look at all the scenting timesteps of a single worker. It stays still for those 6 timesteps, so all same position and bias.

In [3]:
def scenting_of_1_bee_all_t(position_data, replicate_i, worker_i):
    ''' Look at one replicate: a bee's scenting, position, and bias info over 
    all time steps and determine if bee is scenting correctly away from queen 
    at 0,0. Return a list of length of all time steps (e.g. 320) showing T or 
    F for that bee.
    '''
    
#     with open("position_data/{}".format(json_name), "r") as f:
#         position_data = json.load(f)
    
    correct_scenting_1_bee_all_t = []
    CORRECT_SCENTING_THRESHOLD = -0.5

    # k = 0
    for t_i, t_info in enumerate(position_data[replicate_i]["worker_{}".format(worker_i)]):
        scenting = position_data[replicate_i]["worker_{}".format(worker_i)][t_i]["scenting"]

    #     if k > 100:
    #         break

        if scenting == True: 
            bias = (position_data[replicate_i]["worker_{}".format(worker_i)][t_i]["bias_x"], position_data[replicate_i]["worker_{}".format(worker_i)][t_i]["bias_y"])
            # print("bias_xy: {}".format(bias))
            position = (position_data[replicate_i]["worker_{}".format(worker_i)][t_i]["x"], position_data[replicate_i]["worker_{}".format(worker_i)][t_i]["y"])
            # print("position: {}".format(position))

            # Determine if scenting in queen's direction
            r = -np.array(position)         # Negative of vector Worker to Queen
            u_r = r / np.linalg.norm(r)     # Unit vector 

            # Get dot product of direction W->Q unit vector and the unit vector of scenting direction
            similarity = np.dot(u_r, bias)  # bound between -1 and 1
            # print("Similarity: {} \n".format(similarity))

            # If within threshold allowed, bee is scenting correctly
            correct_scenting = True if similarity <= CORRECT_SCENTING_THRESHOLD else False

            # If correct_scenting, add True to correct_scenting_1_bee_all_t, else False
            correct_scenting_1_bee_all_t.append(True) if correct_scenting == True else correct_scenting_1_bee_all_t.append(False)

    #         k += 1

    #         if k % 1 == 0:

    #             plt.title(similarity)
    #             plt.scatter([0], [0], c='k')
    #             plt.scatter(*position, c='r')
    #             plt.quiver(*position, *bias)
    #             plt.quiver(*position, *u_r)
    #             plt.quiver(*position, *-u_r)
    #             plt.xlim([-0.5, 0.5])
    #             plt.ylim([-1, 3]) 
    #             plt.figure()

        else: 
            correct_scenting_1_bee_all_t.append(False)
            
    return correct_scenting_1_bee_all_t

Get correct_scenting_1_bee_all_t lists for all 50 workers in one replicate json. Then average over all 10 replicates to get one cumulative list.

In [4]:
def scenting_of_all_bees_all_t(position_data, rep_i):
    
    ''' Correct scenting info for all bees over all time for a single replicate. 
    Get list of length 50, each list inside is length 320 (all t). 
    Count number of scenting bees at each timestep. Return this list
    of length 320. '''

    all_bees_all_t = []

    for bee_i, bee_info in enumerate(position_data[rep_i]):
        one_bee = scenting_of_1_bee_all_t(position_data, replicate_i=rep_i, worker_i=bee_i+1)
        all_bees_all_t.append(one_bee)
        
    # Count number of correctly scenting bees over time 
    # Combine all separate lists for bees into this list; each list inside is a timestep over all bees
    ### Get scenting for all bees over all time
    over_all_time = []

    # Loop over all timesteps
    for time_i in range(len(all_bees_all_t[0])):
        # print(time_i)
        # Loop over each of 50 lists in all_bees_all_t  
        one_t = []
        for bee_i in range(len(all_bees_all_t)):
            scenting = all_bees_all_t[bee_i][time_i]
            one_t.append(scenting)
        over_all_time.append(one_t)
        
    ### Count scenting bees in swarm over time
    swarm_over_time = []
    for t in over_all_time:
        scenting_true = sum(t)
        swarm_over_time.append(scenting_true)
        
    return swarm_over_time

In [5]:
def scenting_of_all_replicates(json_name):
    
    ''' Correct scenting info for all replicates. 
    Return list of length [number of replicates] '''
    
    with open("position_data/{}".format(json_name), "r") as f:
        position_data = json.load(f)
    
    all_reps = []
    for rep_i in range(len(position_data)):
        rep_lists = scenting_of_all_bees_all_t(position_data, rep_i)
        all_reps.append(rep_lists)
        
    # Write to json
    with open('scenting_correct_data/{}'.format(json_name), 'w') as outfile:
        json.dump(all_reps, outfile)

    return None

In [6]:
# Get data for all sets of params
data_list = list(map(lambda x : x.split("/")[-1], 
                glob2.glob("position_data/*")))
    
for i, d in enumerate(data_list):
    print("{}. Saving correct scenting for: {}".format(i+1, d))
    d_list = scenting_of_all_replicates(d)

1. Saving correct scenting for: Q0.15_W0.0005_D0.005_T0.0050_wb1.json
2. Saving correct scenting for: Q0.15_W0.0005_D0.005_T0.5000_wb1.json


KeyboardInterrupt: 

### WORK FROM HERE: Get proportion of correctly scenting bees

In [7]:
def get_proportions(scenting_all_data, scenting_correct_data, replicate_i):
    
    ''' For one replicate. '''
    
#     with open("scenting_all_data/{}".format(all_json), "r") as f:
#         scenting_all_data = json.load(f)
    
#     with open("scenting_correct_data/{}".format(correct_json), "r") as f:
#         scenting_correct_data = json.load(f)
    
    
    # List of 10 replicates' proportions of correct over t
    correct_proportions = []

    # Over 1 replicate's 320 time steps
    for i in range(len(scenting_all_data[replicate_i])):
        all_scenting = int(scenting_all_data[replicate_i][i])
        correct_scenting = int(scenting_correct_data[replicate_i][i])
        # print(all_scenting, correct_scenting)

        if all_scenting != 0:
            proportion = correct_scenting / all_scenting
            # print(proportion)
            # print("\n")
            correct_proportions.append(proportion * 100)
        else:
            correct_proportions.append(0)

    return correct_proportions

Get all 10 replicates for a param set.

In [8]:
def get_proportions_all_reps(all_json, correct_json):
    
    with open("scenting_all_data/{}".format(all_json), "r") as f1:
        scenting_all = json.load(f1)

    with open("scenting_correct_data/{}".format(correct_json), "r") as f2:
        scenting_correct = json.load(f2)
        
    # Save 10 lists here
    proportions_all_reps = []
    for i in range(len(scenting_all)):
        proportions_1_rep = get_proportions(scenting_all, scenting_correct, i)
        proportions_all_reps.append(proportions_1_rep)
        
    return proportions_all_reps

In [9]:
plt.style.use('seaborn-white')
plt.rcParams['font.sans-serif'] = "Arial"
plt.rcParams['font.size'] = 14
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.labelweight'] = 'normal'
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['figure.titlesize'] = 14
plt.rcParams['lines.linewidth'] = 1
params = {"ytick.color" : "#000000",
          "xtick.color" : "#000000",
          "axes.labelcolor" : "#000000",
          "axes.edgecolor" : "#000000",
          "text.color": "#000000"}
plt.rcParams.update(params)

In [10]:
def standardize_filenames(fname):
    # fname = "Q0.15_W0.0005_D0.5_T0.005_wb1.json"
    split_list = fname.split("_")

    new_name = ""
    for item in split_list:
        if "Q" in item:
            new_name += item + "_"
        elif "W" in item and len(item) < 7:
            new_length = 7 - len(item)
            new_name += item + "0"*new_length + "_"
        elif "D" in item and len(item) < 6:
            new_length = 6 - len(item)
            new_name += item + "0"*new_length + "_"
        elif "T" in item and len(item) < 7:
            new_length = 7 - len(item)
            new_name += item + "0"*new_length + "_"
        elif "wb" in item:
            new_name += item
        else:
            new_name += item + "_"

    return new_name


In [11]:
# Standardize filenames
path = '/Users/dieumynguyen/Desktop/Projects/bee_communication/step4_position_data_scenting/scenting_correct_data'
files = os.listdir(path)
for file in files:
    os.rename(os.path.join(path, file), os.path.join(path, standardize_filenames(file)))


In [19]:
# Get list of files in correct order
all_list = list(map(lambda x : x.split("/")[-1], 
            glob2.glob("scenting_all_data/*T0.5*")))
all_list = all_list[::-1]
updated_all_list = sorted(all_list, key = lambda x: (x.split("_")[1], x[2]))[::-1]

############

correct_list = list(map(lambda x : x.split("/")[-1], 
            glob2.glob("scenting_correct_data/*T0.5*")))
correct_list = correct_list[::-1]
updated_correct_list = sorted(correct_list, key = lambda x: (x.split("_")[1], x[2]))[::-1]


In [ ]:
# Start plotting!
fig, axes = plt.subplots(7, 6, sharex=True, sharey=True, figsize=(30,20))

# Plot subplots
for j, ax in enumerate(axes.flatten()):
    # Get list of 10 lists/replicates
    data = get_proportions_all_reps(updated_all_list[j], updated_correct_list[j])

    # Plotting
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)

    # List of random ints from 0-20
    randoms = random.sample(range(0, 10), 1)
    
    colormap = mpl.cm.Dark2.colors
    # l_styles = ['-','--','-','--','-']
    
    x = np.linspace(0, 320, num=320)
    
    # for i, d in enumerate(data):
    for r in range(len(randoms)):
        # Try on first set 
        ax.plot(x, data[randoms[r]], label='Swarm {}'.format(r+1), color=colormap[r], linestyle="-")
        plt.fill_between(x, data[randoms[r]], alpha=0.8, color=colormap[r])

        plt.xlim(0, 320)
        plt.ylim(0, 110)

        ax.set_title('{}'.format(updated_all_list[j][6:-10]), fontsize=16)

# Add x and y labels for all
ax = fig.add_subplot(111, frameon=False)
ax.set_xticks([])
ax.set_yticks([])
ax.set_xlabel('Time (step)', labelpad=25, fontsize=20) # Use argument `labelpad` to move label downwards.
ax.set_ylabel('Proportion of correctly scenting bees (%)', labelpad=35, fontsize=20)

fig.subplots_adjust(wspace=0.15, hspace=0.3)

# Save figure as a single pdf
plt.savefig("/Users/dieumynguyen/Desktop/Projects/bee_communication/figures/correct_scenting/T0.5_CorrectScenting.pdf", transparent=True)


In [14]:
# def plot_scenting(all_json, correct_json):

#     ''' For each replicate in a set of parameters, open that cumulative
#     JSON and plot each replicate as a line in a single plot. Sample 5 reps.
#     '''
    
#     # Get list of 10 lists/replicates
#     data = get_proportions_all_reps(all_json, correct_json)
    
#     fig = plt.figure(figsize=(15, 5))
#     ax = plt.axes([0.1, 0.1, 0.6, 0.75])  # try

#     # Plotting
#     ax.spines['right'].set_visible(False)
#     ax.spines['top'].set_visible(False)

#     # List of random ints from 0-20
#     randoms = random.sample(range(0, 10), 1)
#     print(randoms)
    
#     colormap = mpl.cm.Dark2.colors
#     # l_styles = ['-','--','-','--','-']
    
#     x = np.linspace(0, 320, num=320)
    
#     # for i, d in enumerate(data):
#     for r in range(len(randoms)):
#         # Try on first set 
#         ax.plot(x, data[randoms[r]], label='Swarm {}'.format(r+1), color=colormap[r], linestyle="-")
#         plt.fill_between(x, data[randoms[r]], alpha=0.8, color=colormap[r])

#         plt.xlim(0, 320)
#         plt.ylim(0, 110)

#         # plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

#         plt.xlabel('Time (step)')
#         plt.ylabel('Proportion of correctly scenting bees (%)')
#         plt.title('{}'.format(correct_json[:-5]))

#         # plt.tight_layout()
#         plt.savefig("/Users/dieumynguyen/Desktop/Projects/bee_communication/figures/correct_scenting/{}.pdf".format(correct_json), transparent=True, bbox_inches='tight')

#     plt.close()


In [25]:
# def main():
#     all_list = list(map(lambda x : x.split("/")[-1], 
#                 glob2.glob("scenting_all_data/*")))
#     correct_list = list(map(lambda x : x.split("/")[-1], 
#                 glob2.glob("scenting_correct_data/*")))
    
#     for i, d in enumerate(all_list):
#         print("{}. Plotting scenting data for: {}".format(i+1, correct_list[i]))
#         d_list = plot_scenting(all_list[i], correct_list[i])
    
# if __name__ == '__main__':
#     main()    